### Login to HuggingFace for downloading a gated model and uploading trained model

In [ ]:
!huggingface-cli login --token your_hf_token_here

### Import Library

In [ ]:
!pip install --upgrade pip
!pip install "huggingface_hub[hf_transfer]"
!pip install git+https://github.com/huggingface/transformers.git -qq
!pip3 install datasets
import torch
import transformers
from tqdm import tqdm
from transformers import AutoTokenizer, TextStreamer
import json
from datasets import load_dataset
import random

Extracting the Instruct Residue and plugging it back to updated base model

In [ ]:
!HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli download meta-llama/Llama-3.2-3B-Instruct
!HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli download meta-llama/Llama-3.2-3B
!HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli download michaelifebrian/Llama-3.2-3B-contpretrain-Medical
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-3.2-3B-Instruct')

pretrainedmodel = transformers.AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B", torch_dtype=torch.bfloat16)
continuedpretrainedmodel = transformers.AutoModelForCausalLM.from_pretrained("michaelifebrian/Llama-3.2-3B-contpretrain-Medical", torch_dtype=torch.bfloat16)
instructresiduemodel = transformers.AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B-Instruct", torch_dtype=torch.bfloat16)

# Extracting Instruct-Residue
with torch.no_grad():
    for instruct, base in tqdm(zip(instructresiduemodel.parameters(), pretrainedmodel.parameters())):
        intruct.data = instruct.data - base.data

# Plugging back the Instruct-Residue to updated base model
with torch.no_grad():
    for copr, inst in tqdm(zip(continuedpretrainedmodel.parameters(), instructresiduemodel.parameters())):
        copr.data = copr.data + inst.data

continuedpretrainedmodel.push_to_hub("Llama-3.2-3B-Updated-InstResi")